In [38]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display

current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
from metrics import *



In [39]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]

## **Best Dropout**

In [40]:


dir_03 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/dropout/03")

dict_03 = {}

detailed_results_df_03 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_03[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_03 = pd.concat([detailed_results_df_03, detailed_df], axis=0)
    detailed_results_df_03.reset_index(drop=True, inplace=True)
detailed_results_df_03.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.226418,-0.083371,0.000000,0.000000,137.0,147.05199,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.112572,0.021719,0.022769,0.021018,145.0,154.43670,1.6,1.476944,A,A,AP,eu,540
2,0,2,-0.027188,0.076729,0.017077,0.011002,151.0,158.30228,1.2,0.773113,A,A,AP,eu,540
3,0,3,0.029735,0.140938,0.011385,0.012842,155.0,162.81430,0.8,0.902406,A,A,AP,eu,540
4,0,4,0.072427,0.160736,0.008538,0.003960,158.0,164.20552,0.6,0.278244,A,A,AP,eu,540


In [41]:
all_03_dfs = list(dict_03.values())

In [42]:
all_03_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_03_dfs]
aggregate_03_df = sum(all_03_value_only_dfs)
# add glycaemic region column
aggregate_03_df["Glycaemic Region"] = all_03_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_03_df = aggregate_03_df[["Glycaemic Region"] + list(aggregate_03_df.columns[:-1])]

aggregate_03_df["AP:EP"] = aggregate_03_df["AP"] / aggregate_03_df["EP"]

aggregate_03_df["AP%"] = aggregate_03_df["AP"] / aggregate_03_df["Count"] * 100
aggregate_03_df["BE%"] = aggregate_03_df["BE"] / aggregate_03_df["Count"] * 100
aggregate_03_df["EP%"] = aggregate_03_df["EP"] / aggregate_03_df["Count"] * 100

display(aggregate_03_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


In [43]:
for ptid, df in dict_03.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_03[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1983,28,202,2213,9.816832,89.606869,1.265251,9.127881
1,eu,29394,2228,154,31776,190.870130,92.503776,7.011581,0.484642
2,hyper,12756,976,1071,14803,11.910364,86.171722,6.593258,7.235020
3,overall,44133,3232,1427,48792,30.927120,90.451303,6.624037,2.924660


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,136,2,22,160,6.181818,85.000000,1.250000,13.750000
1,eu,31050,1125,54,32229,575.000000,96.341804,3.490645,0.167551
2,hyper,15120,600,419,16139,36.085919,93.686102,3.717702,2.596196
3,overall,46306,1727,495,48528,93.547475,95.421200,3.558770,1.020030


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,81,2,13,96,6.230769,84.375000,2.083333,13.541667
1,eu,30087,1787,220,32094,136.759091,93.746495,5.568019,0.685486
2,hyper,25490,1587,829,27906,30.747889,91.342364,5.686949,2.970687
3,overall,55658,3376,1062,60096,52.408663,92.615149,5.617678,1.767173


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1867,127,127,2121,14.700787,88.024517,5.987742,5.987742
1,eu,34968,2616,948,38532,36.886076,90.750545,6.789162,2.460293
2,hyper,14182,957,584,15723,24.284247,90.199071,6.086625,3.714304
3,overall,51017,3700,1659,56376,30.751658,90.494182,6.563076,2.942742


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1222,24,74,1320,16.513514,92.575758,1.818182,5.606061
1,eu,33452,1586,551,35589,60.711434,93.995336,4.456433,1.548231
2,hyper,12728,699,352,13779,36.159091,92.372451,5.072937,2.554612
3,overall,47402,2309,977,50688,48.517912,93.517203,4.555319,1.927478


## **Adjusted Loss Weights**


In [44]:
low_adj_loss_dir = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/loss_function_weights_lowest")
dict_low_adj_loss = {}
detailed_results_df_low_adj_loss = pd.DataFrame()
for ptid in ptid_list:
    df = pd.read_csv(os.path.join(low_adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_low_adj_loss[ptid] = df

    detailed_df = pd.read_csv(os.path.join(low_adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_low_adj_loss = pd.concat([detailed_results_df_low_adj_loss, detailed_df], axis=0)
    detailed_results_df_low_adj_loss.reset_index(drop=True, inplace=True)
detailed_results_df_low_adj_loss.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,1.310500,1.262284,0.000000,0.000000,245.0,241.61183,0.000000,0.000000,A,A,AP,hyper,540
1,0,1,1.282039,1.226730,-0.005692,-0.007111,243.0,239.11343,-0.400000,-0.499679,A,A,AP,hyper,540
2,0,2,1.225116,1.209208,-0.011385,-0.003504,239.0,237.88220,-0.800000,-0.246249,A,A,AP,hyper,540
3,0,3,1.125501,1.204138,-0.019923,-0.001014,232.0,237.52586,-1.400001,-0.071266,A,B,AP,hyper,540
4,0,4,1.054347,1.160455,-0.014231,-0.008736,227.0,234.45627,-1.000000,-0.613918,A,A,AP,hyper,540


In [45]:
all_low_adj_loss_dfs = list(dict_low_adj_loss.values())

In [46]:
all_low_adj_loss_value_only_df = [df.drop(columns=["Glycaemic Region"]) for df in all_low_adj_loss_dfs]
aggregated_low_adj_loss_df = sum(all_low_adj_loss_value_only_df)

aggregate_low_adj_loss_df = aggregated_low_adj_loss_df.copy()
aggregate_low_adj_loss_df["Glycaemic Region"] = all_low_adj_loss_dfs[0]["Glycaemic Region"]

aggregate_low_adj_loss_df = aggregate_low_adj_loss_df[["Glycaemic Region"] + list(aggregate_low_adj_loss_df.columns[:-1])]

aggregate_low_adj_loss_df["AP:EP"] = aggregate_low_adj_loss_df["AP"] / aggregate_low_adj_loss_df["EP"]

aggregate_low_adj_loss_df["AP%"] = aggregate_low_adj_loss_df["AP"] / aggregate_low_adj_loss_df["Count"] * 100
aggregate_low_adj_loss_df["BE%"] = aggregate_low_adj_loss_df["BE"] / aggregate_low_adj_loss_df["Count"] * 100
aggregate_low_adj_loss_df["EP%"] = aggregate_low_adj_loss_df["EP"] / aggregate_low_adj_loss_df["Count"] * 100

display(aggregate_low_adj_loss_df)



,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11883,379,859,13121,13.833527,90.564744,2.888499,6.546757
1,eu,306905,16601,4132,327638,74.275169,93.671979,5.066873,1.261148
2,hyper,172511,9401,6529,188441,26.422270,91.546426,4.988829,3.464745
3,overall,491299,26381,11520,529200,42.647483,92.838057,4.985072,2.176871


In [47]:
for ptid, df in dict_low_adj_loss.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_low_adj_loss[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,2005,29,179,2213,11.201117,90.600994,1.310438,8.088568
1,eu,29483,2140,153,31776,192.699346,92.783862,6.734642,0.481495
2,hyper,12704,1016,1083,14803,11.730379,85.820442,6.863474,7.316085
3,overall,44192,3185,1415,48792,31.231095,90.572225,6.527709,2.900066


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,131,0,29,160,4.517241,81.875000,0.000000,18.125000
1,eu,31071,1104,54,32229,575.388889,96.406963,3.425486,0.167551
2,hyper,15044,657,438,16139,34.347032,93.215193,4.070884,2.713923
3,overall,46246,1761,521,48528,88.763916,95.297560,3.628833,1.073607


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,83,0,13,96,6.384615,86.458333,0.000000,13.541667
1,eu,29987,1836,271,32094,110.653137,93.434910,5.720695,0.844395
2,hyper,25595,1528,783,27906,32.688378,91.718627,5.475525,2.805848
3,overall,55665,3364,1067,60096,52.169634,92.626797,5.597710,1.775493


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1844,116,161,2121,11.453416,86.940123,5.469118,7.590759
1,eu,35042,2533,957,38532,36.616510,90.942593,6.573757,2.483650
2,hyper,14179,999,545,15723,26.016514,90.179991,6.353749,3.466260
3,overall,51065,3648,1663,56376,30.706554,90.579325,6.470839,2.949837


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1188,39,93,1320,12.774194,90.000000,2.954545,7.045455
1,eu,33501,1572,516,35589,64.924419,94.133019,4.417095,1.449886
2,hyper,12718,698,363,13779,35.035813,92.299877,5.065680,2.634444
3,overall,47407,2309,972,50688,48.772634,93.527068,4.555319,1.917614


## **adjusted_Loss**

In [48]:
adj_loss_dir = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/loss_function_weights")
dict_adj_loss = {}
detailed_results_df_adj_loss = pd.DataFrame()
for ptid in ptid_list:
    df = pd.read_csv(os.path.join(adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_adj_loss[ptid] = df

    detailed_df = pd.read_csv(os.path.join(adj_loss_dir, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_adj_loss = pd.concat([detailed_results_df_adj_loss, detailed_df], axis=0)
    detailed_results_df_adj_loss.reset_index(drop=True, inplace=True)
detailed_results_df_adj_loss.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.012957,-0.203157,0.000000,0.000000,152.0,138.63458,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.155264,-0.227507,-0.028461,-0.004870,142.0,136.92351,-2.0,-0.342215,A,B,AP,eu,540
2,0,2,-0.411417,-0.328165,-0.051231,-0.020132,124.0,129.85022,-3.6,-1.414657,A,B,AP,eu,540
3,0,3,-0.596417,-0.435600,-0.037000,-0.021487,111.0,122.30070,-2.6,-1.509905,A,B,AP,eu,540
4,0,4,-0.752955,-0.488805,-0.031308,-0.010641,100.0,118.56192,-2.2,-0.747755,A,B,AP,eu,540


In [49]:
all_adj_loss_dfs = list(dict_adj_loss.values())

In [50]:
all_adj_loss_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_adj_loss_dfs]
aggregate_adj_loss_df = sum(all_adj_loss_value_only_dfs)
# add glycaemic region column
aggregate_adj_loss_df["Glycaemic Region"] = all_adj_loss_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_adj_loss_df = aggregate_adj_loss_df[["Glycaemic Region"] + list(aggregate_adj_loss_df.columns[:-1])]

aggregate_adj_loss_df["AP:EP"] = aggregate_adj_loss_df["AP"] / aggregate_adj_loss_df["EP"]

aggregate_adj_loss_df["AP%"] = aggregate_adj_loss_df["AP"] / aggregate_adj_loss_df["Count"] * 100
aggregate_adj_loss_df["BE%"] = aggregate_adj_loss_df["BE"] / aggregate_adj_loss_df["Count"] * 100
aggregate_adj_loss_df["EP%"] = aggregate_adj_loss_df["EP"] / aggregate_adj_loss_df["Count"] * 100
display(aggregate_adj_loss_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11778,410,933,13121,12.623794,89.764500,3.124762,7.110739
1,eu,306921,16524,4193,327638,73.198426,93.676863,5.043371,1.279766
2,hyper,172614,9426,6401,188441,26.966724,91.601085,5.002096,3.396819
3,overall,491313,26360,11527,529200,42.622799,92.840703,4.981104,2.178193


In [51]:
for ptid, df in dict_adj_loss.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_adj_loss[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1985,27,201,2213,9.875622,89.697244,1.220063,9.082693
1,eu,29437,2178,161,31776,182.838509,92.639099,6.854230,0.506672
2,hyper,12718,1018,1067,14803,11.919400,85.915017,6.876984,7.207998
3,overall,44140,3223,1429,48792,30.888733,90.465650,6.605591,2.928759


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,130,0,30,160,4.333333,81.250000,0.000000,18.750000
1,eu,31067,1097,65,32229,477.953846,96.394551,3.403767,0.201682
2,hyper,15077,672,390,16139,38.658974,93.419667,4.163827,2.416507
3,overall,46274,1769,485,48528,95.410309,95.355259,3.645318,0.999423


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,85,1,10,96,8.500000,88.541667,1.041667,10.416667
1,eu,30080,1768,246,32094,122.276423,93.724684,5.508818,0.766498
2,hyper,25524,1592,790,27906,32.308861,91.464201,5.704866,2.830932
3,overall,55689,3361,1046,60096,53.239962,92.666733,5.592718,1.740548


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1785,149,187,2121,9.545455,84.158416,7.024988,8.816596
1,eu,35054,2541,937,38532,37.410886,90.973736,6.594519,2.431745
2,hyper,14236,965,522,15723,27.272031,90.542517,6.137506,3.319977
3,overall,51075,3655,1646,56376,31.029769,90.597063,6.483255,2.919682


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1194,40,86,1320,13.883721,90.454545,3.030303,6.515152
1,eu,33555,1536,498,35589,67.379518,94.284751,4.315940,1.399309
2,hyper,12641,742,396,13779,31.921717,91.741055,5.385006,2.873939
3,overall,47390,2318,980,50688,48.357143,93.493529,4.573074,1.933396


## **Comparison**

In [52]:
display(aggregate_03_df)

display(aggregate_low_adj_loss_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11883,379,859,13121,13.833527,90.564744,2.888499,6.546757
1,eu,306905,16601,4132,327638,74.275169,93.671979,5.066873,1.261148
2,hyper,172511,9401,6529,188441,26.422270,91.546426,4.988829,3.464745
3,overall,491299,26381,11520,529200,42.647483,92.838057,4.985072,2.176871
